In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from matplotlib import pyplot as plt
import cv2
import numpy as np
import json
import pandas as pd
import imutils
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
config_file = '/mmdetection/configs/detectors/missing_detector_latest_segm.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
checkpoint_file = '/mmdetection/data/missing_model2/epoch_11.pth'

In [3]:
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

2021-04-09 15:28:01,707 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.0.rfp_conv.weight, layer2.0.rfp_conv.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset_l.weight, layer2.1.conv2.offset_l.bias, layer2.2.conv2.weight_diff, layer2.2.conv2.switch.weight, layer2

In [5]:
def remove_list(a,b):
    new_list = [fruit for fruit in a if fruit not in b]
    return new_list

def get_contour(im,out):
    #im=cv2.imread('total2.jpg')
    mask=np.zeros(im.shape[:2])
    pred_mask=out[2]
    for i in range(len(pred_mask)):
        mask=cv2.bitwise_or(mask.astype('uint8'),pred_mask[i].astype('uint8')*255)
    cnt, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)   
    return cnt

def convert_cp_cnt(pnts):
    cnts=[]
    for i in range(int(0.5*len(pnts))):
        cnts.append([[pnts[2*i],pnts[(2*i)+1]]])
    return np.array(cnts,np.int32)

def get_center(c):
    M = cv2.moments(c)
    if M["m00"]==0:
        return np.nan
    else:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return cX,cY

def get_area(c):
    return cv2.contourArea(c)

def get_bbox(c):
    return cv2.boundingRect(c)

def get_rot_bbox(c):
    box = cv2.minAreaRect(c)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    return box

In [6]:
missing_cp_path='missing-carparts-merged.json'
missing_cp_tba_path='missing-carparts-to-be-annotated.json'
img_dir='/mmdetection/data/combine_missing/images/'

with open(missing_cp_path) as f:
    missing_cp=json.load(f)
with open(missing_cp_tba_path) as f:
    missing_cp_tba=json.load(f)

    
carparts_names=['sli_side_turn_light', 'alloy_wheel', 'tail_gate', 'tli_tail_light', 'rwi_rear_windshield', 'fuel_tank_door', 'qpa_quarter_panel', 'lli_low_bumper_tail_light', 'mirror', 'fender', 'hood', 'flp_front_license_plate', 'fbe_fog_light_bezel', 'tyre', 'handle', 'fwi_windshield', 'rbu_rear_bumper', 'window', 'blp_back_license_plate', 'door', 'fbu_front_bumper', 'fli_fog_light', 'roof', 'pillar', 'hli_head_light', 'rocker_panel', 'grille']

column_name=['image_name']

for i in range(4):
    for j in ['area','center','bbox','rot_bbox']:
        for k in carparts_names:
            column_name.append(str(k)+'_'+str(j)+'_'+str(i))

for i in range(5):
    for j in ['area','center','bbox']:
        column_name.append('damage_'+str(j)+'_'+str(i))

column_name.extend(['width','height','total_carparts','label'])

In [23]:

modes=['train','valid']


def get_training_dataset(modes):
    
    df = pd.DataFrame(index=list(range(10000)),columns=column_name)
    row_id=-1
    for mode in modes:

        ann_dir='/mmdetection/data/combine_missing/annotations/missing_'+mode+'.json'
        car_parts_ann='/mmdetection/data/carpart-total/total-missing/tm_'+mode+'_cp_0.json'

        with open(ann_dir) as f:
            data=json.load(f)
        with open(car_parts_ann) as f:
            car_parts_data=json.load(f)

        for i in range(len(data['images'])):
            row_id=row_id+1
            col_out={}
            for cn in column_name:
                col_out[cn]=np.nan
            image_name=data['images'][i]['scalabel_url']
            print(image_name)
            col_out['image_name']=image_name
            width=data['images'][i]['width']
            height=data['images'][i]['height']
            col_out['height']=height
            col_out['width']=width
            fn=data['images'][i]['file_name']
            for j in car_parts_data['predict']:
                #print(j['image_name'])
                if j['image_name']==fn:
                    img=cv2.imread(img_dir+fn)
                    result = inference_detector(model, img_dir+fn)
                    out=show_result_pyplot(model, img_dir+fn, result)
                    if len(out[2])>0:
                        damage_cnt=get_contour(img,out)
                        d_n=0
                        for cd in damage_cnt:
                            col_out['damage_area_'+str(d_n)]=get_area(cd)
                            col_out['damage_center_'+str(d_n)]=get_center(cd)
                            col_out['damage_bbox_'+str(d_n)]=get_bbox(cd)
                            col_out['damage_rot_bbox_'+str(d_n)]=get_rot_bbox(cd)

                    cp_ann=j['damages']
                    for k in cp_ann.keys():
                        cp_ann_ind=cp_ann[k]
                        #print('#'*30)
                        #print(k)
                        #print(len(cp_ann_ind))
                        for m in range(len(cp_ann_ind)):
                            cp_cnt=convert_cp_cnt(cp_ann_ind[m])
                            col_out[k+'_area_'+str(m)]=get_area(cp_cnt)
                            col_out[k+'_center_'+str(m)]=get_center(cp_cnt)
                            col_out[k+'_bbox_'+str(m)]=get_bbox(cp_cnt)
                            #col_out[k+'_rot_bbox_'+str(m)]=get_rot_bbox(cp_cnt)
                            #print(get_rot_bbox(cp_cnt))
                            #print(cp_cnt)
                    col_out['total_carparts']=len(cp_ann.keys())

                    #print(fn)
                    #print(damage_cnt)
            #print(col_out)

            if image_name in missing_cp_tba['image_names']:
                print('TBA')

            if image_name in missing_cp['data'].keys():
                col_out['label']=missing_cp['data'][image_name]
                print('pass')
            else:
                col_out['label']=None
                print('skip')

            df.loc[row_id]=col_out
    df_n=df.dropna(axis = 0, how = 'all')
    df_n=df_n.dropna(subset=['label'])
    
    
    for i in df_n.columns:
        nc_list=[]
        nc_list_c=[]
        if 'center' in i:
            print(i)
            for nc in range(1,3):
                nc_name=i+'_c_'+str(nc)
                nc_list_c.append(nc_name)
                df_n[[nc_name]]=np.nan


            for j in range(len(df_n[[i]])):
                if type(df_n[[i]].iloc[j][0])==str:
                    for fill in range(2):
                        df_n[nc_list_c[fill]].iloc[j]=eval(df_n[[i]].iloc[j][0])[fill]
            df_n=df_n.drop([i], axis=1)
        if 'bbox' in i:
            for nc in range(1,5):
                nc_name=i+'_a_'+str(nc)
                nc_list.append(nc_name)
                df_n[[nc_name]]=np.nan

            for j in range(len(df_n[[i]])):
                if type(df_n[[i]].iloc[j][0])==str:
                    for fill in range(4):
                        df_n[nc_list[fill]].iloc[j]=eval(df_n[[i]].iloc[j][0])[fill]
            df_n=df_n.drop([i], axis=1)

    lb=[]
    lb_check=[]
    print(df_n['label'])
    print(df_n['label'].iloc[0])
    for i in range(len(df_n)):
        #print(df_n['label'].iloc[i])
        #print(df_n['label'].iloc[i])
        #print(type(df_n['label'].iloc[i]))
        df_n['label'].iloc[i]=remove_list(df_n['label'].iloc[i],['front','left','rear','right'])
        lb.append(df_n['label'].iloc[i])
        lb_check.extend(df_n['label'].iloc[i])
        #print(df_n['label'].iloc[i])
        
        
        
    lb=np.array(lb)
    label=df_n[['label']]
    df_n = df_n.drop('label', 1)
    df_n = df_n.drop('image_name', 1)
    #df_n[['sli_side_turn_light_area_0']]=df_n[['sli_side_turn_light_area_0']].astype(float)
    for i in df_n.columns:
        df_n[[i]]=df_n[[i]].astype(float)

    return df_n, lb, label

In [26]:
df_n,lb, label=get_training_dataset(['train','valid'])

https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/Boi%20thuong%20thang%207/PC%2030E%20-%20882.20%20-%2025.07.2018/z1062693702570_c7f4ccf0a5563cf4d4360eff6a08df04.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1314/DSCN6994.JPG
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1311/100_3521.JPG
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019Z6401/front_right_view/28633/medium/9862DAF4-8519-43CC-AD34-3D12F8F089E5.jpeg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/136/WhatsApp+Image+2018-07-27+at+12.57.27+PM+MARK+(4).jpeg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2019A1267/front_left_view/29588/medium/20190909_145505.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/boi%20thuong%20thang%2011.2018/14A%20-%20108.33%20-%2002.11.2018/z1164011539746_c8e16f41efbf7065a2a9fcd010dbfd15.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2018X7016/front_right_view/15533/medium/67EC0DCC-9EB5-4D22-8050-

TBA
skip
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/Boi%20thuong%20thang%2010/PC%2014A%20-%20117.54%20-%2011.10.2018/z1141226931061_13852fb03e96b4ab31ed8128aed2ccc1.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2018X5235/front_left_view/14661/medium/02F7F343-8CFD-4012-A63A-1CE07482B0FF.jpeg
TBA
pass
https://s3.amazonaws.com/mc-ai/imt-attachments/valid_case_before_2018May/2018T2392/vehicle_additional_docs/p6-56.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2019A2578/detail_damage1/30564/medium/20190927_155633_HDR.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019X9932/vehicle_additional_docs/17415/medium/image.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/650/IMG_9950.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/KBC/unzipped/05/44012131826223/img2289290.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1505/IMG_20130531_090651.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020C2135/front_left_vi

TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1023/20130704_133623.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/vn_car/caunghi/cHuyen/ba7793e77f3c361c4dfaf4816a8c69e2.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/boi%20thuong%20thang%2011.2018/TL%2014A%20-%20256.99%20-%2019.11.2018/z1180909477061_71f0d46f136341e7d62b008d8de7e2a5.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/KBC/unzipped/07/44012131906656/img2978097.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/boi%20thuong%20thang%208/Hyundai%2014A%20-%20300.46%20-%2027.08.2018/z1103505025195_a928893d8dafa3beda8b3267b230d6d3.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1314/DSCN7002.JPG
pass
https://s3.amazonaws.com/mc-imt/vehicle/2018X2728/front_right_view/13376/medium/5DF34E9C-25A9-4FDF-B45A-43A49DAE12B7.jpeg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/96/IMG-20181127-WA0093+MARK+(5).jpg
pass
https://s3.amazonaws

TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020C1298/vehicle_additional_docs/45705/medium/20200806_150510.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019Y3377/front_right_view/19390/medium/15524321124931385383266091681977.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2019A3838/vehicle_additional_docs/30917/medium/IMG_20191005_180517727_HDR.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2019A9180/detail_damage1/33994/medium/1578324467534697376103.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/KBC/CZ/images_CSOB/2172000691/315698217_IMG_2675.JPG
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020D0318/vehicle_additional_docs/48876/medium/B629BEFA-1F4D-4B8D-9296-AC465650D7A8.jpeg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2019Y8237/detail_damage1/22589/medium/IMG_20190507_091836.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019Z2238/detail_damage1/24392/medium/20190607_151229.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/201903

TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/805/IMG_5737.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1320/DSCN3256.JPG
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/boi%20thuong%20thang%208/14LD%20-%2000038/z1075735860297_88b11446a1d890dc06cf11eea781c774.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A0320/rear_left_view/30364/medium/20190828_124340.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1138/IMG_20180409_111729.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2020C1677/front_right_view/39447/medium/20200615_122752.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020C6735/vehicle_additional_docs/44240/medium/40BF6FE7-06BC-4BF4-9B83-7900A832BF05.jpeg
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/Boi%20thuong%20thang%2010/LG%2014C%20-%20247.00%20-27.10.2018/z1159210390494_7ce8a172389b26a4e9acb926a2f6c331.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/datas

...

TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019X9445/vehicle_additional_docs/17316/medium/IMG_20190124_143644.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A8441/front_left_view/33245/medium/image.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/762/IMG_6691.JPG
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A0320/detail_damage1/30365/medium/20190828_123347.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A1944/detail_damage1/29944/medium/7D8AC4F7-0759-41C7-BA59-946DE9744FA1.jpeg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/KBC/unzipped/06/44012131827153/img2407714.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/741/IMG_2606.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/boi%20thuong%20thang%208/14LD%20-%2000038/Originals/z1075735862970_e408640dc070a9ec14131cbfeba7d876.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2020B7981/detail_damage1/36981/medium/20200427_084224.jpg
TBA
skip
https

TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2019A4525/detail_damage2/31400/medium/15711825117156107575230080910691.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/KBC/CZ/images_CSOB/2174000255/315373698_11.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/Boi%20thuong%20thang%207/PC%2014A%20-%20195.68%20-%2030.07.2018/1%20%2829%29.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/KBC/CZ/images_CSOB/2174000256/315409849_IMG_0767.JPG
TBA
skip
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_xe/PAGS%2014A%20-%20153.57/1%20%28120%29.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019Z3244/vehicle_additional_docs/24990/medium/EE2633C5-0BA9-4826-9570-C3C8FEB3611F.jpeg
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_xe/HT%2014A%20-%20158.60%20-%2013.12.2018/z1215192705321_5a701b2f129011551b25a6b85b4f72a3.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2020C0064/detail_damage1/38327/medium/image.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/veh

TBA
pass
https://s3.amazonaws.com/mc-ai/salvage_car/frontend/1/0ba5a2050f065fa59afa1208fe535a4e387e9a74.jpg
pass
https://s3.amazonaws.com/mc-ai/salvage_car/frontend/1/0d4da79de7f05121ff72e1ded46031f135f56e32.jpg
pass
https://s3.amazonaws.com/mc-ai/salvage_car/frontend/1/5cdf032607afea95499634e15a76dc1f9bf51ce5.jpg
pass
https://s3.amazonaws.com/mc-imt/vehicle/2018W9661/detail_damage1/11461/medium/IMG_6357.jpg
pass
https://s3.amazonaws.com/mc-ai/salvage_car/frontend/2/32d0a890f3eed646c8dd02259b1864e0b88de017.jpg
pass
https://s3.amazonaws.com/mc-ai/salvage_car/frontend/2/72ebf3db8c2d02d9ce5f3c737e466fcd81b7d0a5.jpg
pass
https://s3.amazonaws.com/mc-imt/vehicle/2018X2327/detail_damage2/13326/medium/15400460179296305645016039709385.jpg
pass
https://s3.amazonaws.com/mc-imt/vehicle/2018W9314/detail_damage2/11426/medium/20180916_170501.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/salvage_car/frontend/1/1d3eb507da9981f50cb626db03dafc57cde98ba3.jpg
pass
https://s3.amazonaws.com/mc-ai/salvage_car/f

https://s3.amazonaws.com/mc-ai/salvage_car/frontend/2/74b859c0f7e36c3abae9c3e28e3613e41400072c.jpg
pass
https://s3.amazonaws.com/mc-imt/vehicle/2018T9768/front_right_view/6283/medium/20180618_161854.jpg
pass
https://s3.amazonaws.com/mc-ai/salvage_car/frontend/1/0a69e8d037e3544bf25bb23784aa1c4b4b0dce91.jpg
pass
sli_side_turn_light_center_0
alloy_wheel_center_0
tail_gate_center_0
tli_tail_light_center_0
rwi_rear_windshield_center_0
fuel_tank_door_center_0
qpa_quarter_panel_center_0
lli_low_bumper_tail_light_center_0
mirror_center_0
fender_center_0
hood_center_0
flp_front_license_plate_center_0
fbe_fog_light_bezel_center_0
tyre_center_0
handle_center_0
fwi_windshield_center_0
rbu_rear_bumper_center_0
window_center_0
blp_back_license_plate_center_0
door_center_0
fbu_front_bumper_center_0
fli_fog_light_center_0
roof_center_0
pillar_center_0
hli_head_light_center_0
rocker_panel_center_0
grille_center_0
sli_side_turn_light_center_1
alloy_wheel_center_1
tail_gate_center_1
tli_tail_light_center

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:
df_n_test,lb_test, label_test=get_training_dataset(['test'])

https://s3.amazonaws.com/mc-imt/vehicle/2019Y7029/front_right_view/21689/medium/39170697-AF14-4983-A7E4-07A0FE3E4DF1.jpeg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2018X0096/front_right_view/12280/medium/20180918_074238.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/imt-attachments/valid_case_before_2018May/2018R8293/rear_right_view/p2-24.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2020C0064/vehicle_additional_docs/38468/medium/732C4B78-6E95-4222-9A17-AA443BA12510.jpeg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020B5480/rear_right_view/34893/medium/C6DF9ED5-676B-4CC5-BACC-D14F39E3767B.jpeg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/615/IMG_5405.JPG
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A2184/vehicle_additional_docs/30029/medium/8DC347F1-2E93-454C-8040-51F8E66D8295.jpeg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A3956/front_right_view/31057/medium/15706333595768738968649374488758.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dat

TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/80/DSCN1009.JPG
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019Y1863/vehicle_additional_docs/28588/medium/E05D339A-ABB7-40BA-882E-A8C8ECE0589D.jpeg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/1132/DSCN2579mark+(2).JPG
TBA
skip
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/boi%20thuong%20thang%209/NA%2014A%20-%20066.66%20-%2006.09.2018/z1106724046954_24540bafac99aab2ae86333a5e3174dd.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020B7981/detail_damage2/36982/medium/20200427_084241.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/KBC/unzipped/08/44012131907644/img2994696.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019Y9200/detail_damage2/22973/medium/387A4578-9646-4D1C-BFC0-D81AA3BDF127.jpeg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/583/IMG_1668.JPG
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/371/DSCN7084.JPG
TBA
skip
https:/

TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019Z2438/front_right_view/24533/medium/20190612_090125.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020B9795/detail_damage2/38086/medium/20200526_185809.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A0305/detail_damage1/29010/medium/image.jpg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/899/IMG_9008.JPG
TBA
skip
https://s3.amazonaws.com/mc-ai/imt-attachments/valid_case_before_2018May/2017R4837/front_left_view/p3-32.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/81/DSCN0115.JPG
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020C9755/vehicle_additional_docs/49108/medium/3ECBB9BD-B2B0-459D-86ED-E447B25A9AB3.jpeg
TBA
skip
https://s3.amazonaws.com/mc-ai/dataset/india/20190312/98/DSCN9303.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/india/20190409/imgs/936/20180315-104325_p10-1843x1382.jpg
TBA
skip
https://s3.amazonaws.com/mc-imt/vehicle/2020C3768/rear_right_view/43790/medium/159

TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/KBC/CZ/images_CSOB/2174000045/315366927_10.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/thinh_anh_3_luu_tru/Boi%20thuong%20thang%2010/PAGS%2014A%20-%20303.02/z1132562310514_4b0cc4cce54af2d674263def03e9ef7a.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2019A6282/vehicle_additional_docs/32176/medium/image.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020C6555/front_left_view/44015/medium/image.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/KBC/CZ/images_CSOB/2174000217/315471654_P1108466.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/dataset/KBC/CZ/images_CSOB/2164029430/315457582_IMG_0066.JPG
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/caunghi/cNghi_Tan_01/50872976_2146887405370892_4414026667073208320_n.jpg
TBA
pass
https://s3.amazonaws.com/mc-ai/vn_car/caunghi/cNghi_Tan_01/50673226_813918422292309_146426305682866176_n.jpg
TBA
pass
https://s3.amazonaws.com/mc-imt/vehicle/2020D4601/vehicle_additional_docs/52601/med

sli_side_turn_light_center_0
alloy_wheel_center_0
tail_gate_center_0
tli_tail_light_center_0
rwi_rear_windshield_center_0
fuel_tank_door_center_0
qpa_quarter_panel_center_0
lli_low_bumper_tail_light_center_0
mirror_center_0
fender_center_0
hood_center_0
flp_front_license_plate_center_0
fbe_fog_light_bezel_center_0
tyre_center_0
handle_center_0
fwi_windshield_center_0
rbu_rear_bumper_center_0
window_center_0
blp_back_license_plate_center_0
door_center_0
fbu_front_bumper_center_0
fli_fog_light_center_0
roof_center_0
pillar_center_0
hli_head_light_center_0
rocker_panel_center_0
grille_center_0
sli_side_turn_light_center_1
alloy_wheel_center_1
tail_gate_center_1
tli_tail_light_center_1
rwi_rear_windshield_center_1
fuel_tank_door_center_1
qpa_quarter_panel_center_1
lli_low_bumper_tail_light_center_1
mirror_center_1
fender_center_1
hood_center_1
flp_front_license_plate_center_1
fbe_fog_light_bezel_center_1
tyre_center_1
handle_center_1
fwi_windshield_center_1
rbu_rear_bumper_center_1
window_

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [66]:
list(mlb.classes_)

['',
 'alloy_wheel',
 'door',
 'fbe_fog_light_bezel',
 'fbu_front_bumper',
 'fender',
 'fli_fog_light',
 'front_license_plate',
 'fuel_tank_door',
 'grille',
 'handle',
 'hli_head_light',
 'hood',
 'hpa_header_panel',
 'lli_low_bumper_tail_light',
 'mirror',
 'qpa_quarter_panel',
 'rbu_rear_bumper',
 'rear_license_plate',
 'rear_windshield',
 'rpa_rear_panel',
 'sli_side_turn_light',
 'tail_gate',
 'tli_tail_light',
 'trunk',
 'tyre',
 'window']

In [81]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(lb)
y_test=MultiLabelBinarizer(classes=mlb.classes_).fit_transform(lb_test)

In [82]:
lb_test

array([list(['fli_fog_light', 'fbe_fog_light_bezel']),
       list(['alloy_wheel', 'tyre', 'handle']), list(['tail_gate']),
       list(['rbu_rear_bumper', 'tail_gate']),
       list(['fbe_fog_light_bezel', 'fli_fog_light']),
       list(['grille', 'hli_head_light']),
       list(['fbu_front_bumper', 'grille', 'fli_fog_light', 'fbe_fog_light_bezel']),
       list(['tli_tail_light', 'rbu_rear_bumper']),
       list(['rbu_rear_bumper']), list(['mirror']), list(['window']),
       list(['grille']), list(['fli_fog_light', 'fbe_fog_light_bezel']),
       list(['', 'fli_fog_light']),
       list(['fli_fog_light', 'fbe_fog_light_bezel']),
       list(['fbe_fog_light_bezel']), list(['grille']),
       list(['grille', 'hli_head_light']),
       list(['fli_fog_light', 'fbe_fog_light_bezel']),
       list(['tyre', 'alloy_wheel', 'handle']),
       list(['window', 'mirror']),
       list(['fli_fog_light', 'fbe_fog_light_bezel']), list(['grille']),
       list(['fli_fog_light']), list(['hli_head_li

In [83]:
lb

array([list(['mirror']),
       list(['rbu_rear_bumper', 'qpa_quarter_panel', 'tyre']),
       list(['mirror']), ...,
       list(['fbu_front_bumper', 'hli_head_light', 'grille', 'fender']),
       list(['hli_head_light']),
       list(['hood', 'fbu_front_bumper', 'hli_head_light', 'grille', 'fender'])],
      dtype=object)

In [84]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [85]:
def get_class_from_index(unique_class,y_test):
    pred_results=[]
    for i in y_test:
        indexes = [j for j,x in enumerate(i) if x == 1]
        class_found=[]
        for k in indexes:
            class_found.append(unique_class[k])
        pred_results.append(class_found)
    return pred_results

In [86]:
get_class_from_index(list(mlb.classes_),y_test)

[['fbe_fog_light_bezel', 'fli_fog_light'],
 ['alloy_wheel', 'handle', 'tyre'],
 ['tail_gate'],
 ['rbu_rear_bumper', 'tail_gate'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['grille', 'hli_head_light'],
 ['fbe_fog_light_bezel', 'fbu_front_bumper', 'fli_fog_light', 'grille'],
 ['rbu_rear_bumper', 'tli_tail_light'],
 ['rbu_rear_bumper'],
 ['mirror'],
 ['window'],
 ['grille'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['', 'fli_fog_light'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['fbe_fog_light_bezel'],
 ['grille'],
 ['grille', 'hli_head_light'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['alloy_wheel', 'handle', 'tyre'],
 ['mirror', 'window'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['grille'],
 ['fli_fog_light'],
 ['hli_head_light'],
 ['hli_head_light'],
 ['rbu_rear_bumper'],
 ['handle'],
 ['hli_head_light'],
 ['lli_low_bumper_tail_light'],
 ['fbe_fog_light_bezel',
  'fbu_front_bumper',
  'fli_fog_light',
  'grille',
  'hli_head_light'],
 ['handle'],
 ['hli_head_light'],
 

In [87]:
get_class_from_index(list(mlb.classes_),y)

[['mirror'],
 ['qpa_quarter_panel', 'rbu_rear_bumper', 'tyre'],
 ['mirror'],
 ['fli_fog_light'],
 ['alloy_wheel', 'tyre'],
 ['hli_head_light'],
 ['fbu_front_bumper', 'grille', 'hood'],
 ['fbu_front_bumper', 'hli_head_light'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['fbe_fog_light_bezel', 'fbu_front_bumper', 'fli_fog_light', 'grille'],
 ['fbe_fog_light_bezel', 'fbu_front_bumper', 'fli_fog_light', 'grille'],
 ['fbe_fog_light_bezel', 'fbu_front_bumper', 'hli_head_light'],
 ['fbe_fog_light_bezel', 'fbu_front_bumper', 'fli_fog_light', 'grille'],
 ['mirror'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['sli_side_turn_light'],
 ['mirror'],
 ['grille', 'hli_head_light', 'hood'],
 ['tli_tail_light'],
 ['hli_head_light'],
 ['fbe_fog_light_bezel',
  'fli_fog_light',
  'front_license_plate',
  'grille',
  'hli_head_light',
  'mirror'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['handle'],
 ['fbe_fog_light_bezel', 'fli_fog_light', 'grille', 'hli_head_light'],
 ['mirror'],
 ['fbe_fog_light_b

In [93]:
clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=10,n_estimators=1000,learning_rate=0.01))

In [94]:
clf.fit(df_n, y)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:44:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:44:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:44:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:44:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.01,
                                            max_delta_step=None, max_depth=10,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=1000, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=None, reg_alpha=None,
                                         

In [110]:
predict=clf.predict(df_n_test)

In [111]:
predict

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [112]:
get_class_from_index(list(mlb.classes_),predict)

[['fbe_fog_light_bezel', 'fli_fog_light'],
 [],
 [],
 [],
 ['fli_fog_light'],
 ['grille'],
 ['fbe_fog_light_bezel', 'fbu_front_bumper', 'fli_fog_light'],
 ['grille'],
 ['grille'],
 [],
 [],
 [],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 [],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['fbe_fog_light_bezel'],
 [],
 ['hli_head_light'],
 [],
 ['handle'],
 ['tli_tail_light'],
 ['fbe_fog_light_bezel', 'fli_fog_light'],
 ['fli_fog_light'],
 [],
 [],
 [],
 [],
 [],
 ['grille'],
 ['alloy_wheel', 'tyre'],
 ['grille', 'hli_head_light'],
 [],
 [],
 ['fbe_fog_light_bezel'],
 ['grille', 'hli_head_light'],
 ['handle', 'mirror'],
 ['tli_tail_light'],
 ['hli_head_light'],
 [],
 [],
 ['mirror'],
 [],
 ['fbe_fog_light_bezel',
  'fbu_front_bumper',
  'fli_fog_light',
  'hli_head_light'],
 [],
 ['grille'],
 ['fbe_fog_light_bezel',
  'fbu_front_bumper',
  'fli_fog_light',
  'grille',
  'hli_head_light',
  'hood'],
 ['hli_head_light', 'mirror'],
 ['rbu_rear_bumper'],
 ['fbe_fog_light_bezel'],
 ['fbe_fog_l